In [5]:
# Importación de bibliotecas necesarias
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

image_path = "/Users/carlosedm10/projects/college/TDSC/Files/i1.png"

In [6]:
# Función para transformar un entero e a un entero positivo e'
def transform_to_positive(e):
    return 2 * e if e >= 0 else 2 * abs(e) - 1


# Codificación Rice-m para un entero e
def rice_encode(e, m):
    e_prime = transform_to_positive(e)
    eq = e_prime // (2**m)  # Cociente
    er = e_prime % (2**m)  # Resto

    # Codificar eq con código unario
    c_eq = "0" * eq + "1"

    # Codificar er con longitud fija de m bits
    c_er = format(er, f"0{m}b")

    return c_eq + c_er


# Decodificación Rice-m para obtener el entero original e
def rice_decode(code, m):
    eq = code.find("1")  # Número de ceros antes del primer uno
    er = int(code[eq + 1 : eq + 1 + m], 2)  # Los m bits siguientes
    e_prime = eq * (2**m) + er

    # Recuperar e del e_prime
    if e_prime % 2 == 0:
        e = e_prime // 2
    else:
        e = -(e_prime + 1) // 2

    return e


# Función para aplicar el predictor de primera diferencia
def predictor_first_difference(x):
    x_pred = np.zeros_like(x)
    x_pred[1:] = x[:-1]
    return x_pred


# Función principal para codificar imágenes predictivamente
def cod_pred(image_path, output_path, m):
    # Cargar la imagen y convertir a escala de grises
    image = Image.open(image_path).convert("L")
    x = np.asarray(image, dtype=np.int32)  # Convertir a array numpy

    # Dimensiones de la imagen
    rows, cols = x.shape

    # Transformar a una secuencia unidimensional (raster)
    x_raster = x.flatten()

    # Aplicar predictor de primera diferencia
    x_pred = predictor_first_difference(x_raster)
    e = x_raster - x_pred  # Error de predicción

    # Codificar los errores de predicción con Rice-m
    bitstream = ""
    for n, e_n in enumerate(e):
        if n == 0:
            # Codificar primera muestra con longitud fija de 8 bits
            bitstream += format(e_n, "08b")
        else:
            # Codificar con Rice-m
            bitstream += rice_encode(e_n, m)

    # Guardar los datos en un archivo binario
    with open(output_path, "wb") as f:
        # Escribir cabecera
        f.write(np.uint16(rows).tobytes())
        f.write(np.uint16(cols).tobytes())
        f.write(np.uint8(m).tobytes())

        # Escribir secuencia de bits
        f.write(int(bitstream, 2).to_bytes((len(bitstream) + 7) // 8, byteorder="big"))

    # Calcular tasa binaria
    nBits = len(bitstream)
    R = nBits / (rows * cols)
    return nBits, R


# Función para encontrar el valor óptimo de m
def find_optimal_m(image_path, m_values):
    best_m = None
    best_rate = float("inf")

    for m in m_values:
        _, R = cod_pred(image_path, f"encoded_m{m}.bin", m)
        print(f"m = {m}, Tasa binaria: {R:.2f} bits/pixel")
        if R < best_rate:
            best_rate = R
            best_m = m

    return best_m, best_rate

In [7]:
m_values = [1, 2, 3, 4, 5]

# Encontrar m óptimo
best_m, best_rate = find_optimal_m(image_path, m_values)
print(f"M óptimo: {best_m}, Tasa binaria mínima: {best_rate:.2f} bits/pixel")

m = 1, Tasa binaria: 10.84 bits/pixel
m = 2, Tasa binaria: 7.20 bits/pixel
m = 3, Tasa binaria: 5.89 bits/pixel
m = 4, Tasa binaria: 5.80 bits/pixel
m = 5, Tasa binaria: 6.31 bits/pixel
M óptimo: 4, Tasa binaria mínima: 5.80 bits/pixel


In [8]:
import numpy as np
from PIL import Image


def codPred(nombreI, nombreS, m):
    # Cargar la imagen y convertirla a escala de grises
    img = Image.open(nombreI).convert("L")
    x = np.array(img, dtype=np.float64)

    # Obtener las dimensiones de la imagen
    nFilas, nColumnas = x.shape

    # Abrir el archivo para escribir los datos
    with open(nombreS, "wb") as f:
        # Escribir las dimensiones y el valor m
        f.write(np.uint16(nFilas).tobytes())
        f.write(np.uint16(nColumnas).tobytes())
        f.write(np.uint8(m).tobytes())

        # Convertir la imagen a un arreglo 1D (Raster)
        x1D = x.T.flatten()

        # Escribir el primer valor de x1D
        f.write(np.uint8(x1D[0]).tobytes())

        nBits = 8  # Inicializamos el contador de bits

        # Procesar los píxeles restantes
        for iPixel in range(1, nFilas * nColumnas):
            e = x1D[iPixel] - x1D[iPixel - 1]
            if e <= 0:
                ep = abs(e) * 2 - 1
            else:
                ep = e * 2

            eq = int(np.floor(ep / 2**m))  # Parte entera
            er = int(np.remainder(ep, 2**m))  # Parte restante

            # Crear el código UNARIO
            cq = np.zeros(eq, dtype=np.uint8)
            cr = np.array([int(x) for x in format(er, f"0{m}b")], dtype=np.uint8)

            # Concatenar los códigos
            c = np.concatenate((cq, cr))

            # Escribir los bits en el archivo
            f.write(c.tobytes())
            nBits += len(c)

        nBits += 35  # Ajustar el contador de bits
        R = nBits / (nFilas * nColumnas)

    return nBits, R